In [1]:
!pip install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Cloning https://github.com/evfro/polara.git (to revision develop) to /private/var/folders/28/jnstkr6902l__8f0j4tmfrwm0000gn/T/pip-install-s_ymx9iv/polara_7646956d89514cfe8eb6dd54961d4b2f
  Running command git clone --filter=blob:none -q https://github.com/evfro/polara.git /private/var/folders/28/jnstkr6902l__8f0j4tmfrwm0000gn/T/pip-install-s_ymx9iv/polara_7646956d89514cfe8eb6dd54961d4b2f
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Resolved https://github.com/evfro/polara.git to commit 4de4ca7d6f901e32f1e045f190bcb09587162397
  Preparing metadata (setup.py) ... done
DEPRECATION: Configuring install

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import polara
from polara import get_movielens_data
from polara.preprocessing.dataframes import leave_one_out, reindex

from dataprep import transform_indices
from evaluation import topn_recommendations, model_evaluate, downvote_seen_items

from polara.lib.tensor import hooi
from polara.lib.sparse import tensor_outer_at

from sa_hooi import sa_hooi, form_attention_matrix, get_scaling_weights, generate_position_projector

from scipy.sparse import csr_matrix

In [2]:
def full_preproccessing():
    data = get_movielens_data(include_time=True)
    test_timepoint = data['timestamp'].quantile(
    q=0.9, interpolation='nearest'
    )

    test_data_ = data.query('timestamp >= @test_timepoint')
    train_data_ = data.query(
    'userid not in @test_data_.userid.unique() and timestamp < @test_timepoint'
    )
    
    training, data_index = transform_indices(train_data_.copy(), 'userid', 'movieid')
    test_data = reindex(test_data_, data_index['items'])

    testset_, holdout_ = leave_one_out(
    test_data, target='timestamp', sample_top=True, random_state=0
    )
    testset_valid_, holdout_valid_ = leave_one_out(
        testset_, target='timestamp', sample_top=True, random_state=0
    )

    test_users_val = np.intersect1d(testset_valid_.userid.unique(), holdout_valid_.userid.unique())
    testset_valid = testset_valid_.query('userid in @test_users_val').sort_values('userid')
    holdout_valid = holdout_valid_.query('userid in @test_users_val').sort_values('userid')

    test_users = np.intersect1d(testset_valid_.userid.unique(), holdout_valid_.userid.unique())
    testset = testset_.query('userid in @test_users').sort_values('userid')
    holdout = holdout_.query('userid in @test_users').sort_values('userid')


    assert holdout_valid.set_index('userid')['timestamp'].ge(
        testset_valid
        .groupby('userid')
        ['timestamp'].max()
    ).all()

    data_description = dict(
        users = data_index['users'].name,
        items = data_index['items'].name,
        feedback = 'rating',
        n_users = len(data_index['users']),
        n_items = len(data_index['items']),
        n_ratings = training['rating'].nunique(),
        min_rating = training['rating'].min()
        #test_users_val = holdout_valid[data_index['users'].name].drop_duplicates().values,
        #test_users_test = holdout[data_index['users'].name].drop_duplicates().values
    )

    return training, testset_valid, holdout_valid, testset, holdout, data_description

In [3]:
training, testset_valid, holdout_valid, testset, holdout, data_description = full_preproccessing()

Filtered 113 invalid observations.


In [4]:
from IPython.utils import io

def tf_model_build(config, data, data_description, attention_matrix=np.array([]), exp_decay = False, decay=1):
    userid = data_description["users"]
    itemid = data_description["items"]
    feedback = data_description["feedback"]

    idx = data[[userid, itemid, feedback]].values
    idx[:, -1] = idx[:, -1] - data_description['min_rating'] # works only for integer ratings!
    val = np.ones(idx.shape[0], dtype='f8')
    
    n_users = data_description["n_users"]
    n_items = data_description["n_items"]
    n_ratings = data_description["n_ratings"]
    shape = (n_users, n_items, n_ratings)
    core_shape = config['mlrank']
    num_iters = config["num_iters"]
    
    if (attention_matrix.shape[0] == 0):
        attention_matrix = form_attention_matrix(
            data_description['n_ratings'],
            **config['params'],
            format = 'csr'
        )

    item_popularity = (
        data[itemid]
        .value_counts(sort=False)
        .reindex(range(n_items))
        .fillna(1)
        .values
    )
    scaling_weights = get_scaling_weights(item_popularity, scaling=config["scaling"])

    with io.capture_output() as captured:
        u0, u1, u2 = sa_hooi(
            idx, val, shape, config["mlrank"],
            attention_matrix = attention_matrix,
            scaling_weights = scaling_weights,
            max_iters = config["num_iters"],
            parallel_ttm = False,
            randomized = config["randomized"],
            growth_tol = config["growth_tol"],
            seed = config["seed"],
            iter_callback = None,
        )
    
    return u0, u1, u2, attention_matrix    
    
config = {
    "scaling": 1,
    "mlrank": (30, 30, 5),
    "n_ratings": data_description['n_ratings'],
    "num_iters": 5,
    "params": None,
    "randomized": True,
    "growth_tol": 1e-4,
    "seed": 42
}


def tf_scoring(params, data, data_description, context=["3+4+5"]):
    user_factors, item_factors, feedback_factors, attention_matrix = params
    userid = data_description["users"]
    itemid = data_description["items"]
    feedback = data_description["feedback"]

    data = data.sort_values(userid)
    useridx = data[userid]
    
    n_users = useridx.nunique()
    n_items = data_description['n_items']
    n_ratings = data_description['n_ratings']
    
    scores = np.zeros((n_users, n_items))
    inv_attention = np.linalg.inv(attention_matrix.A)
    for i, u in tqdm(enumerate(np.unique(useridx))):
        data_u = data[data.userid==u]
        P = csr_matrix((np.ones(data_u.shape[0]), (data_u[itemid].values, data_u[feedback].values - data_description['min_rating'])), (n_items, n_ratings))
        res = item_factors @ (item_factors.T @ (P @ (attention_matrix @ (feedback_factors @ (inv_attention.T @ feedback_factors).T))))
        if (context == "5"):
            scores[i] = np.sum(res[:, -1:], axis=1)
        elif (context == "4+5"):
            scores[i] = np.sum(res[:, -2:], axis=1)
        elif (context == "3+4+5"):
            scores[i] = np.sum(res[:, -3:], axis=1)
        elif (context == "2+3+4+5"):
            scores[i] = np.sum(res[:, -4:], axis=1)
        elif (context == "3+4+5-2-1"):
            scores[i] = np.sum(res[:, 2:], axis=1) - np.sum(res[:, :2], axis=1)
        
    return scores

def model_evaluate(recommended_items, holdout, holdout_description, alpha=3, topn=10):
    itemid = holdout_description['items']
    rateid = holdout_description['feedback']
    holdout_items = holdout[itemid].values
    assert recommended_items.shape[0] == len(holdout_items)
    hits_mask = recommended_items[:, :topn] == holdout_items.reshape(-1, 1)
    # HR calculation
    hr = np.mean(hits_mask.any(axis=1))
    # MRR calculation
    n_test_users = recommended_items.shape[0]
    hit_rank = np.where(hits_mask)[1] + 1.0
    mrr = np.sum(1 / hit_rank) / n_test_users
    # DCG calculation
    pos_mask = (holdout[rateid] >= alpha).values
    neg_mask = (holdout[rateid] < alpha).values
    pos_hit_rank = np.where(hits_mask[pos_mask])[1] + 1.0
    neg_hit_rank = np.where(hits_mask[neg_mask])[1] + 1.0
    ndcg = np.sum(1 / np.log2(pos_hit_rank+1)) / n_test_users
    ndcl = np.sum(1 / np.log2(neg_hit_rank+1)) / n_test_users
    # coverage calculation
    n_items = holdout_description['n_items']
    cov = np.unique(recommended_items).size / n_items
    return hr, mrr, cov, ndcg, ndcl

def make_prediction(tf_scores, holdout, data_description, mode, context=""):
    if (mode):
        print(f"for context {context} evaluation:")
    for n in [5, 10, 20]:
        tf_recs = topn_recommendations(tf_scores, n)
        hr, mrr, cov, dcg, dcl = model_evaluate(tf_recs, holdout_valid, data_description, topn=n)
        print(f"Test : HR@{n} = {hr:.4f}, MRR@{n} = {mrr:.4f}, Coverage@{n} = {cov:.4f}, nDCG@{n} = {dcg}, nDCL@{n} = {dcl}")
        if (n == 10):
            mrr10 = mrr
    return mrr10

def valid_mlrank(mlrank):
    '''
    Only allow ranks that are suitable for truncated SVD computations
    on unfolded compressed tensor (the result of ttm product in HOOI).
    '''
    r1, r2, r3 = mlrank
    return r1*r2 > r3 and r1*r3 > r2 and r2*r3 > r1

In [5]:
from tqdm import tqdm 
from polara.evaluation.pipelines import random_grid

def full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix):

    config["mlrank"] = (30, 30, 5)
    print("Starting pipeline...")
    print("Training with different context in progress...")
    print("------------------------------------------------------")
    best_mrr_context = "3+4+5"
    best_mrr = 0.0
    for context in ["5", "4+5", "3+4+5", "2+3+4+5", "3+4+5-2-1"]:
        tf_params = tf_model_build(config, training, data_description, attention_matrix=attention_matrix)
        seen_data = testset_valid
        tf_scores = tf_scoring(tf_params, seen_data, data_description, context)
        downvote_seen_items(tf_scores, seen_data, data_description)
        cur_mrr = make_prediction(tf_scores, holdout_valid, data_description, "Validation", context)
        print("------------------------------------------------------")
        if (cur_mrr > best_mrr):
            best_mrr = cur_mrr
            best_mrr_context = context

    best_mrr_context = "3+4+5" # intuitively this is better 
    print(f"Tuning model with context {best_mrr_context}...")

    tf_hyper = {
    'r1': range(30, 56, 5),
    'r2': range(30, 56, 5),
    'r3': range(5, 6, 5),
    }

    grid, param_names = random_grid(tf_hyper, n=0)
    tf_grid = [tuple(mlrank) for mlrank in grid if valid_mlrank(mlrank)]

    hr_tf = {}
    mrr_tf = {}
    cov_tf = {}
    for mlrank in tqdm(tf_grid):
        with io.capture_output() as captured:
            config['mlrank'] = mlrank
            tf_params = tf_model_build(config, training, data_description, attention_matrix=attention_matrix)
            tf_scores = tf_scoring(tf_params, seen_data, data_description, best_mrr_context)
            downvote_seen_items(tf_scores, seen_data, data_description)
            tf_recs = topn_recommendations(tf_scores, topn=10)
            hr, mrr, cov, dcg, dcl = model_evaluate(tf_recs, holdout_valid, data_description, topn=10)
            hr_tf[mlrank] = hr
            mrr_tf[mlrank] = mrr
            cov_tf[mlrank] = cov

    print(f'Best HR={pd.Series(hr_tf).max():.4f} achieved with mlrank={pd.Series(hr_tf).idxmax()}')
    print(f'Best MRR={pd.Series(mrr_tf).max():.4f} achieved with mlrank={pd.Series(mrr_tf).idxmax()}')
    print(f'COV={pd.Series(cov_tf)[pd.Series(mrr_tf).idxmax()]:.4f} (based on best HR value)')
    print("---------------------------------------------------------")
    print("Evaluation of the best model on test holdout in progress...")
    
    config["mlrank"] = pd.Series(mrr_tf).idxmax()
    tf_params = tf_model_build(config, training, data_description, attention_matrix=attention_matrix)

    seen_data = testset
    tf_scores = tf_scoring(tf_params, seen_data, data_description, best_mrr_context)
    downvote_seen_items(tf_scores, seen_data, data_description)
    cur_mrr = make_prediction(tf_scores, holdout, data_description, "Test", best_mrr_context)
    print("Pipeline ended.")

## Linear attention, decay factor = 1

In [6]:
linear_attentions_list = [
    {'decay_factor': 1, 'exponential_decay': False, 'reverse': True},
    {'decay_factor': 1, 'exponential_decay': False, 'reverse': False},
]

In [34]:
for params in linear_attentions_list:
    
    config["params"] = params
    
    full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix=np.array([]))

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1238.29it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0361, MRR@5 == 0.0180, Coverage@5 = 0.0883
Validation : HR@10 = 0.0563, MRR@10 == 0.0204, Coverage@10 = 0.1315
Validation : HR@20 = 0.1020, MRR@20 == 0.0235, Coverage@20 = 0.1829
------------------------------------------------------


1137it [00:00, 1242.09it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0255, MRR@5 == 0.0139, Coverage@5 = 0.0913
Validation : HR@10 = 0.0598, MRR@10 == 0.0181, Coverage@10 = 0.1279
Validation : HR@20 = 0.1011, MRR@20 == 0.0211, Coverage@20 = 0.1766
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0264, MRR@5 == 0.0166, Coverage@5 = 0.1128
Validation : HR@10 = 0.0554, MRR@10 == 0.0204, Coverage@10 = 0.1527
Validation : HR@20 = 0.1020, MRR@20 == 0.0236, Coverage@20 = 0.2105
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:38<00:00,  6.08s/it]


Best HR=0.0783 achieved with mlrank=(55, 50, 5)
Best MRR=0.0291 achieved with mlrank=(55, 50, 5)
COV=0.1766 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1060.36it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0281, MRR@5 == 0.0136, Coverage@5 = 0.1307
Test : HR@10 = 0.0545, MRR@10 == 0.0169, Coverage@10 = 0.1774
Test : HR@20 = 0.0994, MRR@20 == 0.0198, Coverage@20 = 0.2415
Pipeline ended.


## Exponential attention, decay factor = 1

In [7]:
exponential_attentions_list = [
    {'decay_factor': 1, 'exponential_decay': True, 'reverse': True},
    {'decay_factor': 1, 'exponential_decay': True, 'reverse': False}
]

In [33]:
for params in exponential_attentions_list:
    
    config["params"] = params
    
    full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix=np.array([]))

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1346.81it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0299, MRR@5 == 0.0162, Coverage@5 = 0.0754
Validation : HR@10 = 0.0431, MRR@10 == 0.0180, Coverage@10 = 0.1111
Validation : HR@20 = 0.0853, MRR@20 == 0.0207, Coverage@20 = 0.1565
------------------------------------------------------


1137it [00:00, 1391.04it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0290, MRR@5 == 0.0133, Coverage@5 = 0.0779
Validation : HR@10 = 0.0545, MRR@10 == 0.0166, Coverage@10 = 0.1092
Validation : HR@20 = 0.0897, MRR@20 == 0.0188, Coverage@20 = 0.1538
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0317, MRR@5 == 0.0163, Coverage@5 = 0.1073
Validation : HR@10 = 0.0519, MRR@10 == 0.0189, Coverage@10 = 0.1469
Validation : HR@20 = 0.0932, MRR@20 == 0.0218, Coverage@20 = 0.2072
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [04:02<00:00,  6.74s/it]


Best HR=0.0712 achieved with mlrank=(55, 45, 5)
Best MRR=0.0260 achieved with mlrank=(55, 45, 5)
COV=0.1678 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1129.78it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0229, MRR@5 == 0.0121, Coverage@5 = 0.1241
Test : HR@10 = 0.0493, MRR@10 == 0.0157, Coverage@10 = 0.1673
Test : HR@20 = 0.1003, MRR@20 == 0.0191, Coverage@20 = 0.2261
Pipeline ended.


## Eucledian distance attention

In [8]:
eucl_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        eucl_matrix[i, j] = abs(i - j) / np.exp(abs(i - j)) if i != j else 5 + 1e-2
        
a = np.linalg.cholesky(eucl_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)

In [12]:
eucl_matrix

array([[5.01      , 0.36787944, 0.27067057, 0.14936121, 0.07326256],
       [0.36787944, 5.01      , 0.36787944, 0.27067057, 0.14936121],
       [0.27067057, 0.36787944, 5.01      , 0.36787944, 0.27067057],
       [0.14936121, 0.27067057, 0.36787944, 5.01      , 0.36787944],
       [0.07326256, 0.14936121, 0.27067057, 0.36787944, 5.01      ]])

In [36]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1284.75it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0299, MRR@5 == 0.0192, Coverage@5 = 0.1051
Validation : HR@10 = 0.0554, MRR@10 == 0.0224, Coverage@10 = 0.1458
Validation : HR@20 = 0.0941, MRR@20 == 0.0251, Coverage@20 = 0.2044
------------------------------------------------------


1137it [00:01, 1122.81it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0273, MRR@5 == 0.0175, Coverage@5 = 0.1073
Validation : HR@10 = 0.0607, MRR@10 == 0.0218, Coverage@10 = 0.1439
Validation : HR@20 = 0.1020, MRR@20 == 0.0247, Coverage@20 = 0.2008
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0009, MRR@5 == 0.0003, Coverage@5 = 0.1631
Validation : HR@10 = 0.0009, MRR@10 == 0.0003, Coverage@10 = 0.2308
Validation : HR@20 = 0.0044, MRR@20 == 0.0005, Coverage@20 = 0.3252
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:51<00:00,  6.44s/it]


Best HR=0.0800 achieved with mlrank=(50, 50, 5)
Best MRR=0.0296 achieved with mlrank=(50, 50, 5)
COV=0.1766 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1043.65it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0317, MRR@5 == 0.0143, Coverage@5 = 0.1285
Test : HR@10 = 0.0589, MRR@10 == 0.0177, Coverage@10 = 0.1769
Test : HR@20 = 0.0967, MRR@20 == 0.0201, Coverage@20 = 0.2358
Pipeline ended.


## Rating distribution attention

In [9]:
rating_dist = []

total_cnt = training.shape[0]

for i in range(5):
    val = training.query(f'rating == {i + 1}').count()[0] / total_cnt
    
    rating_dist.append(val)

rating_dist

[0.056644931644931645,
 0.10500693000693001,
 0.25943898443898444,
 0.3442087192087192,
 0.2347004347004347]

In [15]:
np.round(rat_dist_matrix, 2)

array([[1.1 , 0.05, 0.17, 0.22, 0.15],
       [0.05, 1.1 , 0.13, 0.19, 0.11],
       [0.17, 0.13, 1.1 , 0.08, 0.02],
       [0.22, 0.19, 0.08, 1.1 , 0.1 ],
       [0.15, 0.11, 0.02, 0.1 , 1.1 ]])

In [10]:
rat_dist_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        diff = abs(rating_dist[i] - rating_dist[j])
        rat_dist_matrix[i, j] = diff / np.exp(diff) if i != j else 1. + 1e-1
        
a = np.linalg.cholesky(rat_dist_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)
a

array([[ 1.00000000e-05,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 4.39343845e-02,  1.00000000e-05,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.57865247e-01,  1.19666749e-01,  1.00000000e-05,
         0.00000000e+00,  0.00000000e+00],
       [ 2.05660310e-01,  1.71084548e-01,  2.42154277e-02,
         1.00000000e-05,  0.00000000e+00],
       [ 1.42079211e-01,  1.02755253e-01, -1.02838852e-02,
         5.08948653e-02,  1.00000000e-05]])

In [93]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1578.35it/s]


for context 5 evaluation:
Test : HR@5 = 0.0299, MRR@5 = 0.0164, Coverage@5 = 0.0949, nDCG@5 = 0.01974361059223822, nDCL@5 = 0.0
Test : HR@10 = 0.0554, MRR@10 = 0.0195, Coverage@10 = 0.1337, nDCG@10 = 0.027115814538717472, nDCL@10 = 0.0005780450156306099
Test : HR@20 = 0.0880, MRR@20 = 0.0217, Coverage@20 = 0.1796, nDCG@20 = 0.034121561049943884, nDCL@20 = 0.0017234280095878207
------------------------------------------------------


1137it [00:00, 1228.88it/s]


for context 4+5 evaluation:
Test : HR@5 = 0.0317, MRR@5 = 0.0188, Coverage@5 = 0.0988, nDCG@5 = 0.021991341338205694, nDCL@5 = 0.0
Test : HR@10 = 0.0598, MRR@10 = 0.0226, Coverage@10 = 0.1309, nDCG@10 = 0.030445577120236732, nDCL@10 = 0.0006064560426045343
Test : HR@20 = 0.0950, MRR@20 = 0.0250, Coverage@20 = 0.1772, nDCG@20 = 0.03823413443949537, nDCL@20 = 0.0017425388095942825
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Test : HR@5 = 0.0325, MRR@5 = 0.0203, Coverage@5 = 0.1004, nDCG@5 = 0.02330381527815138, nDCL@5 = 0.0
Test : HR@10 = 0.0624, MRR@10 = 0.0244, Coverage@10 = 0.1304, nDCG@10 = 0.032442689825717075, nDCL@10 = 0.0006265737680000391
Test : HR@20 = 0.1029, MRR@20 = 0.0272, Coverage@20 = 0.1769, nDCG@20 = 0.04104534979727037, nDCL@20 = 0.002209530709621398
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:40<00:00,  6.12s/it]


Best HR=0.0800 achieved with mlrank=(45, 45, 5)
Best MRR=0.0279 achieved with mlrank=(40, 45, 5)
COV=0.1560 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1024.14it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0000, MRR@5 = 0.0000, Coverage@5 = 0.1161, nDCG@5 = 0.0, nDCL@5 = 0.0
Test : HR@10 = 0.0000, MRR@10 = 0.0000, Coverage@10 = 0.1549, nDCG@10 = 0.0, nDCL@10 = 0.0
Test : HR@20 = 0.0000, MRR@20 = 0.0000, Coverage@20 = 0.2077, nDCG@20 = 0.0, nDCL@20 = 0.0
Pipeline ended.


## Trigonometry scale attention

In [7]:
def rescale_score(x, func=None):
    
    if func is None:
        func = np.arctan
    
    return func(x)

In [8]:
np.round(eucl_matrix, 2)

NameError: name 'eucl_matrix' is not defined

In [9]:
eucl_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        
        k, l = rescale_score(i + 1), rescale_score(j + 1)
        
        diff = abs(k - l)
        
        eucl_matrix[i, j] = diff / np.exp(diff) if i != j else 5 + 1e-2
        
a = np.linalg.cholesky(eucl_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)

In [43]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1441.94it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0343, MRR@5 == 0.0208, Coverage@5 = 0.1048
Validation : HR@10 = 0.0624, MRR@10 == 0.0244, Coverage@10 = 0.1461
Validation : HR@20 = 0.1038, MRR@20 == 0.0271, Coverage@20 = 0.2014
------------------------------------------------------


1137it [00:01, 1129.38it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0308, MRR@5 == 0.0183, Coverage@5 = 0.1103
Validation : HR@10 = 0.0633, MRR@10 == 0.0228, Coverage@10 = 0.1475
Validation : HR@20 = 0.1108, MRR@20 == 0.0260, Coverage@20 = 0.2003
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0009, MRR@5 == 0.0003, Coverage@5 = 0.1653
Validation : HR@10 = 0.0018, MRR@10 == 0.0004, Coverage@10 = 0.2360
Validation : HR@20 = 0.0026, MRR@20 == 0.0005, Coverage@20 = 0.3249
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:58<00:00,  6.62s/it]


Best HR=0.0844 achieved with mlrank=(50, 50, 5)
Best MRR=0.0303 achieved with mlrank=(50, 50, 5)
COV=0.1750 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1045.84it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0237, MRR@5 == 0.0103, Coverage@5 = 0.1287
Test : HR@10 = 0.0589, MRR@10 == 0.0150, Coverage@10 = 0.1744
Test : HR@20 = 0.0994, MRR@20 == 0.0177, Coverage@20 = 0.2338
Pipeline ended.


In [10]:
def center_and_rescale_score(x, func=None):
    
    if func is None:
        func = np.arctan
    
    return func(x - 3)

In [13]:
np.round(similarity)

array([[1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0.],
       [0., 1., 1., 1., 0.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.]])

In [12]:
eucl_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        
        k, l = center_and_rescale_score(i + 1), center_and_rescale_score(j + 1)
        
        diff = abs(k - l)
        
        eucl_matrix[i, j] = 1 / (diff + 1)

similarity = eucl_matrix
        
print(similarity)
    
a = np.linalg.cholesky(similarity)        

attention_matrix = csr_matrix(a)

[[1.         0.75657241 0.47457495 0.34571609 0.31110998]
 [0.75657241 1.         0.56009915 0.38898453 0.34571609]
 [0.47457495 0.56009915 1.         0.56009915 0.47457495]
 [0.34571609 0.38898453 0.56009915 1.         0.75657241]
 [0.31110998 0.34571609 0.47457495 0.75657241 1.        ]]


In [50]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1482.42it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0290, MRR@5 == 0.0190, Coverage@5 = 0.1111
Validation : HR@10 = 0.0528, MRR@10 == 0.0222, Coverage@10 = 0.1530
Validation : HR@20 = 0.0871, MRR@20 == 0.0245, Coverage@20 = 0.2168
------------------------------------------------------


1137it [00:00, 1424.28it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0281, MRR@5 == 0.0192, Coverage@5 = 0.1103
Validation : HR@10 = 0.0598, MRR@10 == 0.0235, Coverage@10 = 0.1505
Validation : HR@20 = 0.1108, MRR@20 == 0.0270, Coverage@20 = 0.2039
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0299, MRR@5 == 0.0180, Coverage@5 = 0.1109
Validation : HR@10 = 0.0572, MRR@10 == 0.0217, Coverage@10 = 0.1527
Validation : HR@20 = 0.1099, MRR@20 == 0.0253, Coverage@20 = 0.2116
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:38<00:00,  6.07s/it]


Best HR=0.0827 achieved with mlrank=(50, 50, 5)
Best MRR=0.0293 achieved with mlrank=(55, 50, 5)
COV=0.1816 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1105.85it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0246, MRR@5 == 0.0129, Coverage@5 = 0.1312
Test : HR@10 = 0.0554, MRR@10 == 0.0168, Coverage@10 = 0.1816
Test : HR@20 = 0.0985, MRR@20 == 0.0198, Coverage@20 = 0.2454
Pipeline ended.


## Conditional prob attention

In [70]:
train_new_part = train_new.query('rating == 1| rating == 2')
users = train_new_part.userid.unique()
count12_tot = 0
count12_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 2:
     count12_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count12_loc += 1
  count12_tot += count12_loc

In [71]:
train_new_part = train_new.query('rating == 1| rating == 3')
users = train_new_part.userid.unique()
count13_tot = 0
count13_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 3:
     count13_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count13_loc += 1
  count13_tot += count13_loc

In [72]:
train_new_part = train_new.query('rating == 1| rating == 4')
users = train_new_part.userid.unique()
count14_tot = 0
count14_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 4:
     count14_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count14_loc += 1
  count14_tot += count14_loc

In [73]:
train_new_part = train_new.query('rating == 1| rating == 5')
users = train_new_part.userid.unique()
count15_tot = 0
count15_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count15_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count15_loc += 1
  count15_tot += count15_loc

In [74]:
train_new_part = train_new.query('rating == 2| rating == 3')
users = train_new_part.userid.unique()
count23_tot = 0
count23_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 3:
     count23_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 2:
      count23_loc += 1
  count23_tot += count23_loc

In [75]:
train_new_part = train_new.query('rating == 2| rating == 4')
users = train_new_part.userid.unique()
count24_tot = 0
count24_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 4:
     count24_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 2:
      count24_loc += 1
  count24_tot += count24_loc

In [76]:
train_new_part = train_new.query('rating == 2| rating == 5')
users = train_new_part.userid.unique()
count25_tot = 0
count25_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count25_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 2:
      count25_loc += 1
  count25_tot += count25_loc

In [77]:
train_new_part = train_new.query('rating == 3| rating == 4')
users = train_new_part.userid.unique()
count34_tot = 0
count34_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 4:
     count34_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 3:
      count34_loc += 1
  count34_tot += count34_loc

In [78]:
train_new_part = train_new.query('rating == 3| rating == 5')
users = train_new_part.userid.unique()
count35_tot = 0
count35_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count35_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 3:
      count35_loc += 1
  count35_tot += count35_loc

In [79]:
train_new_part = train_new.query('rating == 4| rating == 5')
users = train_new_part.userid.unique()
count45_tot = 0
count45_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count45_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 4:
      count45_loc += 1
  count45_tot += count45_loc

In [80]:
train_new_part = train_new.query('rating == 1')
users = train_new_part.userid.unique()
count11_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count11_tot += len(train_new_part_user)-1

In [81]:
train_new_part = train_new.query('rating == 2')
users = train_new_part.userid.unique()
count22_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count22_tot += len(train_new_part_user)-1

In [82]:
train_new_part = train_new.query('rating == 3')
users = train_new_part.userid.unique()
count33_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count33_tot += len(train_new_part_user)-1

In [83]:
train_new_part = train_new.query('rating == 4')
users = train_new_part.userid.unique()
count44_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count44_tot += len(train_new_part_user)-1

In [84]:
train_new_part = train_new.query('rating == 5')
users = train_new_part.userid.unique()
count55_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count55_tot += len(train_new_part_user)-1

In [85]:
rat_dist_matrix = np.zeros((5, 5))

rat_dist_matrix[0][0] = count11_tot
rat_dist_matrix[0][1] = rat_dist_matrix[1][0]= count12_tot
rat_dist_matrix[0][2] = rat_dist_matrix[2][0]= count13_tot
rat_dist_matrix[0][3] = rat_dist_matrix[3][0]= count14_tot
rat_dist_matrix[0][4] = rat_dist_matrix[4][0]= count15_tot
rat_dist_matrix[1][1] = count22_tot
rat_dist_matrix[1][2] = rat_dist_matrix[2][1] = count23_tot
rat_dist_matrix[1][3] = rat_dist_matrix[3][1] = count24_tot
rat_dist_matrix[1][4] = rat_dist_matrix[4][1] = count25_tot
rat_dist_matrix[2][2] = count33_tot
rat_dist_matrix[2][3] = rat_dist_matrix[3][2] = count34_tot
rat_dist_matrix[2][4] = rat_dist_matrix[4][2] = count35_tot
rat_dist_matrix[3][3] = count44_tot
rat_dist_matrix[3][4] = rat_dist_matrix[4][3] = count45_tot
rat_dist_matrix[4][4] = count55_tot       

In [86]:
summ = np.sum(rat_dist_matrix)
rat_dist_matrix /= summ

In [87]:
rat_dist_matrix[0,0] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[1,1] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[2,2] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[3,3] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[4,4] += np.random.uniform(low=0.0, high=1.0)

In [88]:
a = np.linalg.cholesky(rat_dist_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)

In [89]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1542.55it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0299, MRR@5 == 0.0164, Coverage@5 = 0.0949
Validation : HR@10 = 0.0554, MRR@10 == 0.0195, Coverage@10 = 0.1337
Validation : HR@20 = 0.0880, MRR@20 == 0.0217, Coverage@20 = 0.1796
------------------------------------------------------


1137it [00:00, 1359.75it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0317, MRR@5 == 0.0188, Coverage@5 = 0.0988
Validation : HR@10 = 0.0598, MRR@10 == 0.0226, Coverage@10 = 0.1309
Validation : HR@20 = 0.0950, MRR@20 == 0.0250, Coverage@20 = 0.1772
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0325, MRR@5 == 0.0203, Coverage@5 = 0.1004
Validation : HR@10 = 0.0624, MRR@10 == 0.0244, Coverage@10 = 0.1304
Validation : HR@20 = 0.1029, MRR@20 == 0.0272, Coverage@20 = 0.1769
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:55<00:00,  6.54s/it]


Best HR=0.0800 achieved with mlrank=(45, 45, 5)
Best MRR=0.0279 achieved with mlrank=(40, 45, 5)
COV=0.1560 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1100.26it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0290, MRR@5 == 0.0126, Coverage@5 = 0.1161
Test : HR@10 = 0.0528, MRR@10 == 0.0157, Coverage@10 = 0.1549
Test : HR@20 = 0.1038, MRR@20 == 0.0190, Coverage@20 = 0.2077
Pipeline ended.
